<h3> Palabras (TF-IDF)</h3>
<ol><li>Obtener el vector del centroide: centroide = centroides[cluster_label]</li>
<li>Obtener los índices de las n palabras con valores más altos en el centroide
        palabras_representativas_indices = np.argsort(centroide)[::-1][:n]</li></ol>
<h3>Documentos (TF-IDF) a nivel de clúster</h3>
<ol><li>Calcular Tf a nivel de clúster</li>
<li>Calcular IDF a nivel de clúster</li>
<li>Generar puntuaciones para cada documento del clúster </li></ol>
<h3> Palabras (Word2Vec)</h3>
<ol><li>Obtener el vector del centroide: centroide = centroides[cluster_label]</li>
<li>Encontrar las palabras más cercanas al centroide en el modelo Word2Vec
        palabras_cercanas = model_w2v.wv.most_similar(positive=[centroide], topn=n)</li></ol>
<h3> Documentos (Word2Vec)</h3>
    <ol><li> Se utiliza la misma funcion que para TF-IDF</li>
</ol>

In [1]:
import pandas as pd
import pickle
import bz2
from IPython.display import display, HTML
import numpy as np
import ast
from gensim.corpora import Dictionary
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
with open('recursos/resultados_modelos.pkl', 'rb') as archivo:
    resultados = pickle.load(archivo)
    
# Bigramas
# with open('resultados_modelos_bi.pkl', 'rb') as archivo:
#     resultados = pickle.load(archivo)

In [5]:
lista_resultados = resultados["kmeans_Tfidf"]['resultados_ejecuciones']

# Crear un DataFrame de pandas
df = pd.DataFrame(lista_resultados)

# Eliminar la columna 'confusion_matrix' ya que no se puede representar directamente en HTML
df = df.drop(columns=['confusion_matrix'])

# Convertir el DataFrame a HTML
tabla_html = df.to_html(index=False)

# Imprimir la tabla HTML
display(HTML(tabla_html))

adjusted_rand_score,normalized_mutual_info_score,fowlkes_mallows_score,bcubed_precision,bcubed_recall,bcubed_f1
0.099076,0.393186,0.161949,0.449619,0.166230,0.170452
0.116662,0.394557,0.180705,0.449484,0.172255,0.182132
0.097570,0.395459,0.161686,0.445950,0.172077,0.177371
0.106154,0.398824,0.171312,0.445362,0.185530,0.185768
0.095128,0.390598,0.158885,0.442554,0.164820,0.171750


In [6]:
# Calcular la media para cada fila
medias = df[['adjusted_rand_score', 'normalized_mutual_info_score', 'bcubed_f1']].mean(axis=1)

# Añadir las medias como una nueva columna al DataFrame
df['Media'] = medias

df_ordenado = df.sort_values(by='Media',ascending=False)

# Crear un nuevo dataframe solo con las medias
df_medias = df_ordenado[['Media']]

# Convertir el DataFrame a HTML
tabla_html_medias = df_medias.to_html()

display(HTML(tabla_html_medias))

,Media
1,0.231117
3,0.230249
2,0.223467
0,0.220904
4,0.219158


In [3]:
etiquetas_predichas=resultados["kmeans_Tfidf"]['etiquetas_predichas'][1]

In [4]:
with open('recursos/modelos_topicos/kmeans_Tfidf_ejecucion_123.pkl', 'rb') as archivo:
    modelo = pickle.load(archivo)
# with bz2.BZ2File('recursos/modelos_topicos_bi/kmeans_Tfidf_ejecucion_123.pkl.bz2', 'rb') as f:
#     modelo = pickle.load(f)

C:\Users\Runob\anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator KMeans from version 1.6.1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
def filter_documents_for_dictionary(tokenized_documents, filtered_words):
    filtered_documents = []
    for doc in tokenized_documents:
        filtered_doc = [word for word in doc if word in filtered_words]
        filtered_documents.append(filtered_doc)
    return filtered_documents

In [6]:
#df_preprocesado = pd.read_csv('df_preprocesado.csv', sep=';')
dictionary = Dictionary.load('recursos/dictionary_filtrado_spacy.gensim')
filtered_words = set(dictionary.token2id.keys())
n_term=len(filtered_words)
#preprocesado_documents=df_preprocesado['texto_preprocesado'].apply(ast.literal_eval)    
#documents = filter_documents_for_dictionary(preprocesado_documents, filtered_words) 
with open('recursos/documents.pkl', 'rb') as archivo:
    documents = pickle.load(archivo)

In [26]:
documentos_como_cadenas = [" ".join(doc) for doc in documents]

In [7]:
vectorizer = TfidfVectorizer()
matriz_tfidf = vectorizer.fit_transform(documentos_como_cadenas)

In [8]:
vocabulario = vectorizer.get_feature_names_out()

In [9]:
from collections import Counter

def obtener_clusters_ordenados(etiquetas):
 
    # Contar el número de documentos por clúster
    conteo_clusters = Counter(etiquetas)

    # Eliminar el clúster residual (-1)
    del conteo_clusters[-1]

    # Ordenar por número de documentos (de mayor a menor)
    clusters_ordenados = sorted(conteo_clusters.items(), key=lambda x: x[1], reverse=True)

    return clusters_ordenados

In [14]:
# Uso de la función
# Asumiendo que `etiquetas` contiene las etiquetas generadas por el modelo KMeans
clusters_ordenados = obtener_clusters_ordenados(etiquetas_predichas)
print(clusters_ordenados)


[(26, 2515), (7, 2346), (43, 2253), (12, 2142), (59, 1972), (46, 1881), (24, 1754), (41, 1738), (25, 1656), (35, 1641), (4, 1555), (23, 1529), (37, 1480), (27, 1443), (54, 1408), (18, 1338), (29, 1317), (1, 1311), (33, 1281), (17, 1251), (48, 1242), (8, 1224), (30, 1184), (42, 1174), (50, 1157), (45, 1148), (39, 1146), (56, 1100), (34, 1065), (15, 1064), (44, 1040), (52, 1023), (53, 991), (13, 976), (55, 972), (3, 936), (38, 925), (9, 872), (6, 862), (20, 849), (51, 830), (0, 823), (2, 820), (57, 817), (60, 730), (58, 725), (28, 707), (49, 652), (40, 610), (31, 602), (14, 591), (36, 560), (16, 532), (21, 529), (32, 487), (10, 415), (5, 365), (11, 362), (22, 297), (47, 150), (19, 90)]


In [21]:
import numpy as np
import pandas as pd

def obtener_palabras_representativas_por_cluster(centroides, vocabulario, etiquetas, n=10, n_top_clusters=20, output_excel_path="palabras_representativas_clusters.xlsx"):
  
    # Obtener los clústeres ordenados por el número de documentos
    clusters_ordenados = obtener_clusters_ordenados(etiquetas)

    # Tomar los top N clústeres
    top_clusters = clusters_ordenados[:n_top_clusters]

    # Crear una lista para almacenar los datos de las palabras y su clúster
    data = []

    # Iterar sobre los clústeres seleccionados
    for cluster_label, _ in top_clusters:
        # Obtener el vector del centroide
        centroide = centroides[cluster_label]

        # Obtener los índices de las n palabras con valores más altos en el centroide
        palabras_representativas_indices = np.argsort(centroide)[::-1][:n]

        # Obtener las palabras representativas
        palabras_representativas = [vocabulario[i] for i in palabras_representativas_indices]

        # Unir las palabras representativas en una sola cadena separada por comas
        palabras_string = ", ".join(palabras_representativas)

        # Añadir el ID del clúster y las palabras al listado
        data.append({"Cluster ID": cluster_label, "Palabras": palabras_string})

    # Crear un DataFrame de pandas
    df = pd.DataFrame(data)

    # Exportar a Excel
    df.to_excel(output_excel_path, index=False)

    print(f"Las palabras representativas de los clústeres se han exportado a '{output_excel_path}'.")

# Uso de la función
# Asumiendo que `centroides`, `vocabulario`, y `etiquetas` están disponibles



In [23]:
centroides = modelo.cluster_centers_
obtener_palabras_representativas_por_cluster(centroides, vocabulario, etiquetas_predichas,n=15)

Las palabras representativas de los clústeres se han exportado a 'palabras_representativas_clusters.xlsx'.


In [23]:
import numpy as np
import pandas as pd
from collections import defaultdict
from numpy.linalg import norm

def imprimir_documentos_relevantes_clusters(etiquetas_predichas, documentos_preprocesados, documentos_originales, top_index, num_documentos=5, nombre_archivo="documentos_relevantes.xlsx"):
       
    resultados_totales = []
 
    for cluster_id in top_index:
        documentos_cluster = [(i, documentos_preprocesados[i]) for i, etiqueta in enumerate(etiquetas_predichas) if etiqueta == cluster_id]

        # Calcular TF
        tf_cluster = defaultdict(int)
        for _, documento in documentos_cluster:
            for palabra in documento:
                tf_cluster[palabra] += 1

        # Calcular IDF
        palabras_por_cluster = defaultdict(set)
        for i, etiqueta in enumerate(etiquetas_predichas):
            palabras_por_cluster[etiqueta].update(documentos_preprocesados[i])

        num_clusters = len(set(etiquetas_predichas))
        num_clusters_con_palabra = {}
        idf_cluster = {}
        peso_palabras = {}
        for palabra in tf_cluster:
            contador = 0
            for palabras in palabras_por_cluster.values():
                if palabra in palabras:
                    contador += 1
            num_clusters_con_palabra[palabra] = contador
            frecuencia = num_clusters_con_palabra[palabra]
            idf = np.log((1 + num_clusters) / (1 + frecuencia)) + 1
            idf_cluster[palabra] = idf
            peso_palabras[palabra] = tf_cluster[palabra] * idf_cluster[palabra]

        # Evaluar documentos del clúster
        puntuaciones_relevancia = []
        for doc_id, documento in documentos_cluster:
            # Calcular la frecuencia relativa de cada palabra
            total_palabras_doc = len(documento)
            puntuacion = np.array([peso_palabras.get(palabra, 0) * (documento.count(palabra) / total_palabras_doc) for palabra in documento])
            puntuacion_final = sum(puntuacion)

            # Cortar los primeros 300 caracteres del documento original
            documento_recortado = documentos_originales[doc_id][:300]

            puntuaciones_relevancia.append((doc_id, cluster_id, documento_recortado, puntuacion_final))

        # Ordenar por puntuación
        puntuaciones_relevancia_ordenadas = sorted(puntuaciones_relevancia, key=lambda x: x[3], reverse=True)[:num_documentos]

        # Añadir resultados de este cluster
        resultados_totales.extend(puntuaciones_relevancia_ordenadas)

    # Verificar que hay datos antes de crear el DataFrame
    if resultados_totales:
        df = pd.DataFrame(resultados_totales, columns=["ID", "ID de Cluster", "Documento (200 caracteres)", "Puntuación"])

        # Guardar en formato CSV, XLSX o ODS según el nombre de archivo
        if nombre_archivo.endswith('.csv'):
            df.to_csv(nombre_archivo, index=False, encoding="utf-8")
        elif nombre_archivo.endswith('.ods'):
            df.to_excel(nombre_archivo, index=False, engine="odf")
        else:
            df.to_excel(nombre_archivo, index=False, engine="openpyxl")
        print(f"Archivo guardado: {nombre_archivo}")
    else:
        print("No hay documentos relevantes para los clusters indicados.")


In [ ]:
#documentos_originales=df_preprocesado['texto']
with bz2.BZ2File('recursos/documentos_originales.pkl.bz2', 'rb') as f:
    documentos_originales = pickle.load(f)
# Obtener los clústeres ordenados por el número de documentos
clusters_ordenados = obtener_clusters_ordenados(etiquetas_predichas)

# Tomar los top N clústeres
top_clusters = clusters_ordenados[:20]
top_clusters_ids = [cluster_id for cluster_id, _ in top_clusters]

In [47]:
top_clusters_ids

[30, 4, 3, 18, 1, 32, 13, 16, 40, 37, 21, 10, 23, 20, 9, 28, 27, 34, 59, 22]

In [ ]:
imprimir_documentos_relevantes_clusters(etiquetas_predichas, documents, documentos_originales, top_clusters_ids, num_documentos=5, nombre_archivo="documentos_relevantes_kmeans_tfidf.xlsx")

# Word2vec

In [4]:
with open('recurso/resultados_modelos.pkl', 'rb') as archivo:
    resultados = pickle.load(archivo)

In [5]:
lista_resultados = resultados["kmeans_word2vec_250"]['resultados_ejecuciones']

#lista_resultados = resultados["kmeans_word2vec_100"]['resultados_ejecuciones'] Para bigramas


# Crear un DataFrame de pandas
df = pd.DataFrame(lista_resultados)

# Eliminar la columna 'confusion_matrix' ya que no se puede representar directamente en HTML
df = df.drop(columns=['confusion_matrix'])

# Convertir el DataFrame a HTML
tabla_html = df.to_html(index=False)

# Imprimir la tabla HTML
display(HTML(tabla_html))

adjusted_rand_score,normalized_mutual_info_score,fowlkes_mallows_score,bcubed_precision,bcubed_recall,bcubed_f1
0.138535,0.420432,0.216817,0.471478,0.145074,0.181187
0.130193,0.416937,0.207329,0.468674,0.141263,0.173602
0.122815,0.417390,0.198260,0.470868,0.136772,0.167968
0.127586,0.416247,0.203817,0.469811,0.141752,0.171377
0.128947,0.416050,0.204385,0.465154,0.144416,0.172522


In [6]:
# Calcular la media para cada fila
medias = df[['adjusted_rand_score', 'normalized_mutual_info_score', 'bcubed_f1']].mean(axis=1)

# Añadir las medias como una nueva columna al DataFrame
df['Media'] = medias

df_ordenado = df.sort_values(by='Media',ascending=False)

# Crear un nuevo dataframe solo con las medias
df_medias = df_ordenado[['Media']]

# Convertir el DataFrame a HTML
tabla_html_medias = df_medias.to_html()

display(HTML(tabla_html_medias))

,Media
0,0.246718
1,0.240244
4,0.239173
3,0.238403
2,0.236058


In [5]:
etiquetas_predichas=resultados["kmeans_word2vec_250"]['etiquetas_predichas'][0]
# etiquetas_predichas=resultados["kmeans_word2vec_100"]['etiquetas_predichas'][1] Para bigramas


In [6]:
with open('resultado/modelos_topicos/kmeans_word2vec_250_ejecucion_42.pkl', 'rb') as archivo:
    modelo = pickle.load(archivo)
# Para bigramas
# with open('resultado/modelos_topicos_bi/kmeans_word2vec_100_ejecucion_123.pkl', 'rb') as archivo:
#     modelo = pickle.load(archivo)

In [9]:
def obtener_palabras_representativas_kmeans_w2v2(model_w2v, labels, centroides, n=15,n_top_clusters=20,output_excel_path="palabras_representativas_kmeans_w2v.xlsx"):
 
    # Obtener los clústeres ordenados por el número de documentos
    clusters_ordenados = obtener_clusters_ordenados(labels)

    # Tomar los top N clústeres
    top_clusters = clusters_ordenados[:n_top_clusters]
    cluster_palabras_representativas = {}
# Crear una lista para almacenar los datos de las palabras y su clúster
    data = []
    for cluster_label, _ in top_clusters:
        # Obtener el centroide del clúster
        centroide = centroides[cluster_label]

        # Encontrar las palabras más cercanas al centroide en el modelo Word2Vec
        palabras_cercanas = model_w2v.wv.most_similar(positive=[centroide], topn=n)

        cluster_palabras_representativas[cluster_label] = [palabra for palabra, _ in palabras_cercanas]

       # Unir las palabras representativas en una sola cadena separada por comas
        palabras_string = ", ".join(cluster_palabras_representativas[cluster_label])

        # Añadir el ID del clúster y las palabras al listado
        data.append({"Cluster ID": cluster_label, "Palabras": palabras_string})

    # Crear un DataFrame de pandas
    df = pd.DataFrame(data)

    # Exportar a Excel
    df.to_excel(output_excel_path, index=False)

    print(f"Las palabras representativas de los clústeres se han exportado a '{output_excel_path}'.")

In [14]:
clusters_ordenados = obtener_clusters_ordenados(etiquetas_predichas)
top_clusters = clusters_ordenados[:20]

In [12]:
 # Cargar el modelo Word2Vec y las etiquetas predichas

modelo_w2v_cargado = Word2Vec.load("recurso/w2v_250_epochs.model")
centroides = modelo.cluster_centers_

# Obtener las palabras representativas
palabras_representativas = obtener_palabras_representativas_kmeans_w2v2(modelo_w2v_cargado, etiquetas_predichas, centroides)


Las palabras representativas de los clústeres se han exportado a 'palabras_representativas_kmeans_w2v.xlsx'.


In [10]:
# df_preprocesado = pd.read_csv('df_preprocesado.csv', sep=';')
# documentos_originales=df_preprocesado['texto']
with bz2.BZ2File('recursos/documentos_originales.pkl.bz2', 'rb') as f:
    documentos_originales = pickle.load(f)
# Obtener los clústeres ordenados por el número de documentos
clusters_ordenados = obtener_clusters_ordenados(etiquetas_predichas)

# Tomar los top N clústeres
top_clusters = clusters_ordenados[:20]
top_clusters_ids = [cluster_id for cluster_id, _ in top_clusters]

In [ ]:
imprimir_documentos_relevantes_clusters(etiquetas_predichas, documents, documentos_originales, top_clusters_ids, num_documentos=5, nombre_archivo="documentos_relevantes_kmeans_word2vecv1.xlsx")